<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [40]:

class D:
    def __init__(self):
        self.classvars = {}
    def __set_name__(self, cls, name):
        self.name = name
    def __get__(self, instance, ownerclass):
        if instance:
            if isinstance(instance, type):
                return self.classvars[instance]
            return instance.__dict__.get(self.name, None)
        return ownerclass.__dict__[self.name]
    def __set__(self, instance, val):
        if isinstance(instance, type):
            self.classvars[instance] = val
        else:
            instance.__dict__[self.name] = val

class Class(type): 
    def __new__(cls, name, base, clsdict):
        for att in clsdict:
            if att.startswith('_'): continue
            if not isinstance(clsdict[att], D):
                raise TypeError("Attribute must be of type 'Att'")
        def fset(instance, att, val):
            #print("%s.__setattr__(fset)" % type(instance).__name__, instance, att, val)
            if not hasattr(type(instance), att):
                m = "Class instance '%s' has no Att '%s'"
                raise AttributeError(m % (instance, att))
            super(type(instance), instance).__setattr__(att, val)
        clsdict['__setattr__'] = fset
        return type.__new__(cls, name, base, clsdict)
    
    def __setattr__(cls, att, val):
        #print("Class.__setattr__", cls, att, val)
        if not hasattr(type(cls), att):
            m = "Class type '%s' has no Att '%s'"
            raise AttributeError(m % (cls, att))
        super().__setattr__(att, val)

In [41]:

class YT(Class, metaclass=Class):
    d = D()
    
class Y(metaclass=YT): pass
class X(metaclass=YT): 
    a = D()

Y.d = 1
X.d = 2

YT.__setattr__(fset) <class '__main__.Y'> d 1
Class.__setattr__ <class '__main__.Y'> d 1
YT.__setattr__(fset) <class '__main__.X'> d 2
Class.__setattr__ <class '__main__.X'> d 2


In [42]:
Y.d, X.d

(1, 2)

In [43]:
YT.d.classvars

{__main__.X: 2, __main__.Y: 1}

In [44]:
try:
    Y.x = 12
except AttributeError as e:
    print(e)

YT.__setattr__(fset) <class '__main__.Y'> x 12
Class instance '<class '__main__.Y'>' has no Att 'x'


In [45]:
try:
    YT.a = 12
except AttributeError as e:
    print(e)

Class.__setattr__ <class '__main__.YT'> a 12
Class type '<class '__main__.YT'>' has no Att 'a'


In [46]:
try:
    YT.d = 1
except AttributeError as e:
    print(e)

Class.__setattr__ <class '__main__.YT'> d 1
Class type '<class '__main__.YT'>' has no Att 'd'


In [47]:
x = X()
try:
    x.z = 12
except AttributeError as e:
    print(e)

X.__setattr__(fset) <__main__.X object at 0x000002A0D828C940> z 12
Class instance '<__main__.X object at 0x000002A0D828C940>' has no Att 'z'


In [48]:
x.a = 12
x.a

X.__setattr__(fset) <__main__.X object at 0x000002A0D828C940> a 12


12

In [49]:
try:
    class Z(metaclass=Class):
        i = 1
except TypeError as e:
    print(e)

att must be of type 'D'
